# Улучшения моделя

Цель данного этапа — улучшить качество базовой модели классификации
тональности текстовых отзывов IMDb за счёт архитектурных улучшений
и подбора гиперпараметров.

На данном этапе данные и процедура предобработки считаются фиксированными.
Все изменения касаются исключительно архитектуры модели и её параметров.


In [34]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pickle
from pathlib import Path
import optuna

c:\Users\nurs\PycharmProjects\pythonProject5\nlp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


###  Загрузка подготовленных данных и артефактов

На данном этапе мы загружаем результаты предыдущих шагов:
- подготовленные тензоры train / validation / test
- словарь word2idx
- фиксируем размер словаря

In [5]:
ARTIFACTS_DIR = Path("artifacts")
MODELS_DIR = Path("models")

In [6]:
data = torch.load(ARTIFACTS_DIR / "dataset.pt")
X_train = data["X_train"]
y_train = data["y_train"]
X_val = data["X_val"]
y_val = data["y_val"]
X_test = data["X_test"]
y_test = data["y_test"]

C:\Users\nurs\AppData\Local\Temp\ipykernel_18456\2661674695.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(ARTIFACTS_DIR / "dataset.pt")


In [ ]:
with open(ARTIFACTS_DIR / "word2idx.pkl", "rb") as f:
    word2idx = pickle.load(f)
VOCAB_SIZE = len(word2idx)
PAD_IDX = word2idx["<PAD>"]

### Создание Dataset и DataLoader

Для обучения и оценки модели используются PyTorch Dataset и DataLoader.
Это позволяет:
- эффективно загружать данные батчами
- использовать перемешивание для train выборки
- обеспечить единый интерфейс для обучения и валидации

In [8]:
class IMDBTensorDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return self.X.size(0)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [9]:
BATCH_SIZE = 64
train_dataset = IMDBTensorDataset(X_train, y_train)
val_dataset   = IMDBTensorDataset(X_val, y_val)
test_dataset  = IMDBTensorDataset(X_test, y_test)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
batch_X, batch_y = next(iter(train_loader))
batch_X.shape, batch_y.shape

(torch.Size([64, 400]), torch.Size([64]))

## Наша базовая модель

In [12]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, padding_idx):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=padding_idx
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        hidden = hidden.squeeze(0)
        logits = self.fc(hidden)
        return logits.squeeze(1)

In [13]:
EMBEDDING_DIM = 128
HIDDEN_DIM = 128
baseline_model = LSTMClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    padding_idx=PAD_IDX
)
baseline_model.load_state_dict(
    torch.load(MODELS_DIR / "baseline_lstm.pt", map_location=device)
)
baseline_model = baseline_model.to(device)
baseline_model.eval()

C:\Users\nurs\AppData\Local\Temp\ipykernel_18456\659086518.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(MODELS_DIR / "baseline_lstm.pt", map_location=devi

LSTMClassifier(
  (embedding): Embedding(20000, 128, padding_idx=0)
  (lstm): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [14]:
with torch.no_grad():
    logits = baseline_model(batch_X.to(device))
logits.shape

torch.Size([64])

In [16]:
def compute_accuracy(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)
            probs = torch.sigmoid(logits)
            preds = (probs >= 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    return correct / total


In [17]:
val_accuracy_baseline = compute_accuracy(baseline_model, val_loader, device)
test_accuracy_baseline = compute_accuracy(baseline_model, test_loader, device)
val_accuracy_baseline, test_accuracy_baseline

(0.5045333333333333, 0.5068)

###  Архитектура модели с Masked Pooling и Dropout

В отличие от базовой BiLSTM, здесь используется:
- masked mean pooling
- masked max pooling
- dropout для регуляризации

Это позволяет учитывать информацию по всей последовательности
и снижает влияние padding-токенов.

In [18]:
class BiLSTMClassifier(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        hidden_dim,
        padding_idx
    ):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=padding_idx
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        forward_hidden = hidden[0]
        backward_hidden = hidden[1]
        combined = torch.cat(
            (forward_hidden, backward_hidden),
            dim=1
        )
        logits = self.fc(combined)
        return logits.squeeze(1)

###  Обучение модели и выбор лучшего чекпойнта

Модель обучается в течение нескольких эпох.
Лучший чекпойнт выбирается на основе минимального значения validation loss,
что позволяет избежать переобучения.

In [19]:
model = BiLSTMClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    padding_idx=PAD_IDX
)
model = model.to(device)
model

BiLSTMClassifier(
  (embedding): Embedding(20000, 128, padding_idx=0)
  (lstm): LSTM(128, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [22]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            total_loss += loss.item()

    return total_loss / len(dataloader)


In [ ]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for X_batch, y_batch in dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


In [ ]:
criterion = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 
EPOCHS = 5
best_val_loss = float("inf")
best_state = None
best_epoch = None

EPOCHS = 5

for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(
        model, train_loader, optimizer, criterion, device
    )

    val_loss = evaluate(
        model, val_loader, criterion, device
    )

    val_acc = compute_accuracy(
        model, val_loader, device
    )

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = {
            k: v.detach().cpu().clone()
            for k, v in model.state_dict().items()
        }
        best_epoch = epoch

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )
print(f"\nBest epoch: {best_epoch} | Best val loss: {best_val_loss:.4f}")

Epoch 01 | Train Loss: 0.2343 | Val Loss: 0.3959 | Val Acc: 0.8492
Epoch 02 | Train Loss: 0.1791 | Val Loss: 0.4031 | Val Acc: 0.8563
Epoch 03 | Train Loss: 0.1865 | Val Loss: 0.4901 | Val Acc: 0.8127
Epoch 04 | Train Loss: 0.1396 | Val Loss: 0.4272 | Val Acc: 0.8643
Epoch 05 | Train Loss: 0.1083 | Val Loss: 0.4373 | Val Acc: 0.8529

Best epoch: 1 | Best val loss: 0.3959


In [26]:
model.load_state_dict(best_state)
model.to(device)

BiLSTMClassifier(
  (embedding): Embedding(20000, 128, padding_idx=0)
  (lstm): LSTM(128, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

### Шаг 5. Оценка качества модели на test выборке

После обучения загружается лучший чекпойнт,
и вычисляется итоговая accuracy на test данных.


In [27]:
test_accuracy_best = compute_accuracy(
    model,
    test_loader,
    device
)
print(f"Test Accuracy (BiLSTM, best checkpoint): {test_accuracy_best:.4f}")

Test Accuracy (BiLSTM, best checkpoint): 0.8472


## BiLSTM с Masked Pooling и Dropout 
Основные идеи данного шага:
- учитывать вклад каждого слова в отзыве, а не только крайние токены
- полностью игнорировать padding-токены при агрегации признаков
- снизить переобучение с помощью dropout

Для этого используются masked mean pooling и masked max pooling,
которые позволяют сформировать устойчивое и информативное
представление всего текста. BiLSTM с Masked Pooling и Dropout

In [ ]:
class BiLSTMPoolingClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, padding_idx, dropout=0.5):
        super().__init__()

        self.padding_idx = padding_idx

        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=padding_idx
        )

        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 4, 1)

    def forward(self, x):
        """
        x: (batch, seq_len)
        """
        embedded = self.embedding(x)              
        output, _ = self.lstm(embedded)         
        mask = (x != self.padding_idx)              
        mask = mask.unsqueeze(-1)                    
        output_masked = output * mask           
        lengths = mask.sum(dim=1).clamp(min=1)       
        mean_pool = output_masked.sum(dim=1) / lengths  
        output_for_max = output.masked_fill(~mask, -1e9)
        max_pool = output_for_max.max(dim=1).values  
        pooled = torch.cat([mean_pool, max_pool], dim=1)  

        pooled = self.dropout(pooled)
        logits = self.fc(pooled).squeeze(1)        

        return logits

In [29]:
POOL_DROPOUT = 0.5

model_pool = BiLSTMPoolingClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    padding_idx=PAD_IDX,
    dropout=POOL_DROPOUT
).to(device)

with torch.no_grad():
    logits = model_pool(batch_X.to(device))

logits.shape


torch.Size([64])

In [30]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_pool.parameters(), lr=1e-3)

best_val_loss = float("inf")
best_state = None
best_epoch = None

EPOCHS = 5

for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(model_pool, train_loader, optimizer, criterion, device)
    val_loss   = evaluate(model_pool, val_loader, criterion, device)
    val_acc    = compute_accuracy(model_pool, val_loader, device)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = {k: v.detach().cpu().clone() for k, v in model_pool.state_dict().items()}
        best_epoch = epoch

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )

print(f"\nBest epoch: {best_epoch} | Best val loss: {best_val_loss:.4f}")


Epoch 01 | Train Loss: 0.4575 | Val Loss: 0.3208 | Val Acc: 0.8637
Epoch 02 | Train Loss: 0.2609 | Val Loss: 0.2684 | Val Acc: 0.8917
Epoch 03 | Train Loss: 0.2051 | Val Loss: 0.2699 | Val Acc: 0.8884
Epoch 04 | Train Loss: 0.1487 | Val Loss: 0.2805 | Val Acc: 0.8933
Epoch 05 | Train Loss: 0.1012 | Val Loss: 0.3011 | Val Acc: 0.8963

Best epoch: 2 | Best val loss: 0.2684


## Оценка модели BiLSTM с Masked Pooling и Dropout

In [31]:
model_pool.load_state_dict(best_state)
model_pool.to(device)
test_acc = compute_accuracy(model_pool, test_loader, device)
print(f"Test Accuracy (BiLSTM + Pooling + Dropout, best): {test_acc:.4f}")

Test Accuracy (BiLSTM + Pooling + Dropout, best): 0.9015


В результате применения данной архитектуры была достигнута
accuracy около 0.90 на test выборке,
что значительно превосходит результаты базовой BiLSTM модели.

## Подбор гиперпараметров (Optuna)

После архитектурных улучшений следующим этапом является
подбор гиперпараметров модели.

Цель данного шага — найти оптимальную конфигурацию модели,
которая обеспечивает наилучшее качество на validation выборке.


In [ ]:
def objective(trial):
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])
    dropout = trial.suggest_float("dropout", 0.2, 0.6)
    lr = trial.suggest_float("lr", 1e-4, 3e-3, log=True)

    model = BiLSTMPoolingClassifier(
        vocab_size=VOCAB_SIZE,
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=hidden_dim,
        padding_idx=PAD_IDX,
        dropout=dropout
    ).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    EPOCHS = 3 

    for _ in range(EPOCHS):
        train_one_epoch(model, train_loader, optimizer, criterion, device)

    val_acc = compute_accuracy(model, val_loader, device)
    return val_acc

In [35]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)

[I 2025-12-16 17:57:59,352] A new study created in memory with name: no-name-38c30e6e-aa12-4369-acb0-ff228df5e133
[I 2025-12-16 17:58:21,145] Trial 0 finished with value: 0.8332 and parameters: {'hidden_dim': 64, 'dropout': 0.5679973463480082, 'lr': 0.0002835043270899687}. Best is trial 0 with value: 0.8332.
[I 2025-12-16 17:58:42,181] Trial 1 finished with value: 0.7876 and parameters: {'hidden_dim': 64, 'dropout': 0.377379684563925, 'lr': 0.00012118126112518238}. Best is trial 0 with value: 0.8332.
[I 2025-12-16 18:01:16,947] Trial 2 finished with value: 0.8569333333333333 and parameters: {'hidden_dim': 128, 'dropout': 0.23135716278985619, 'lr': 0.0002450578989315937}. Best is trial 2 with value: 0.8569333333333333.
[I 2025-12-16 18:03:31,141] Trial 3 finished with value: 0.8970666666666667 and parameters: {'hidden_dim': 64, 'dropout': 0.4576022212944503, 'lr': 0.002167795446235008}. Best is trial 3 with value: 0.8970666666666667.
[I 2025-12-16 18:14:03,217] Trial 4 finished with val

### Результаты оптимизации Optuna

In [36]:
study.best_trial


FrozenTrial(number=4, state=<TrialState.COMPLETE: 1>, values=[0.9077333333333333], datetime_start=datetime.datetime(2025, 12, 16, 18, 3, 31, 142415), datetime_complete=datetime.datetime(2025, 12, 16, 18, 14, 3, 217092), params={'hidden_dim': 256, 'dropout': 0.41375295777293747, 'lr': 0.0012688497338454597}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'hidden_dim': CategoricalDistribution(choices=(64, 128, 256)), 'dropout': FloatDistribution(high=0.6, log=False, low=0.2, step=None), 'lr': FloatDistribution(high=0.003, log=True, low=0.0001, step=None)}, trial_id=4, value=None)

In [37]:
print("Best validation accuracy:", study.best_value)
print("Best hyperparameters:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")


Best validation accuracy: 0.9077333333333333
Best hyperparameters:
  hidden_dim: 256
  dropout: 0.41375295777293747
  lr: 0.0012688497338454597


В результате оптимизации был найден следующий набор
гиперпараметров, обеспечивший наилучшее качество
на validation выборке:

- hidden_dim — размер скрытого состояния LSTM
- dropout — коэффициент dropout для регуляризации
- learning rate — скорость обучения оптимизатора

### Финальное обучение модели с оптимальными параметрами

In [38]:
best_params = study.best_params
final_model = BiLSTMPoolingClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=best_params["hidden_dim"],
    padding_idx=PAD_IDX,
    dropout=best_params["dropout"]
).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])
EPOCHS = 5
best_val_loss = float("inf")
best_state = None
for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(final_model, train_loader, optimizer, criterion, device)
    val_loss   = evaluate(final_model, val_loader, criterion, device)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = {k: v.detach().cpu().clone() for k, v in final_model.state_dict().items()}
    print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
final_model.load_state_dict(best_state)
final_model.to(device)


Epoch 01 | Train Loss: 0.4229 | Val Loss: 0.2895
Epoch 02 | Train Loss: 0.2371 | Val Loss: 0.2512
Epoch 03 | Train Loss: 0.1663 | Val Loss: 0.2365
Epoch 04 | Train Loss: 0.1109 | Val Loss: 0.2811
Epoch 05 | Train Loss: 0.0652 | Val Loss: 0.2972


BiLSTMPoolingClassifier(
  (embedding): Embedding(20000, 128, padding_idx=0)
  (lstm): LSTM(128, 256, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.41375295777293747, inplace=False)
  (fc): Linear(in_features=1024, out_features=1, bias=True)
)

Выше представлена финальная архитектура модели
после применения архитектурных улучшений
и подбора гиперпараметров.

Модель включает:
- embedding слой
- двунаправленную LSTM с увеличенным hidden_dim
- masked mean и max pooling
- dropout для регуляризации
- линейный классификатор

### Финальная оценка модели на test выборке

In [39]:
final_test_acc = compute_accuracy(final_model, test_loader, device)
print(f"Final Test Accuracy (Optuna tuned): {final_test_acc:.4f}")

Final Test Accuracy (Optuna tuned): 0.9117


## Сохранение нашей улучшенной модели

In [ ]:
torch.save(
    final_model.state_dict(),
    "artifacts/bilstm_pooling_optuna_best.pt"
)

### Финальный вывод по этапу Model Tuning

В ходе этапа Model Tuning были выполнены:
- архитектурные улучшения модели
- устранение влияния padding-токенов с помощью masked pooling
- регуляризация с использованием dropout
- автоматический подбор гиперпараметров с помощью Optuna

Финальная модель достигла accuracy 0.91 на test выборке,
что значительно превосходит результаты базовой модели
и подтверждает эффективность выбранного подхода.
